In [2]:
import numpy as np
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml
from scipy.optimize import fmin_l_bfgs_b


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
Data, labels = ml.loadFile('./Train.txt')
n, m = sum(labels == 1), sum(labels == 0)
#print('Prima ', n/m)
numpy.random.seed(0)
indexes = numpy.random.permutation(1200)
Data_s = Data[:, indexes]
label_s = labels[indexes]
ns, ms = sum(label_s == 1), sum(label_s == 0)
#print('Dopo :', ns/ms)

In [4]:
(DTR, LTR), (DTE, LTE) = ml.split_db_2to1(Data_s, label_s, seed=42)
n1, n0 = sum(LTR == 1), sum(LTR == 0)

In [9]:
def L_dual_wrapper(H_hat):
    def L_dual(alpha):
        one_vect = numpy.ones((len(alpha)), dtype='int32')
        L_d = 1/2 * numpy.dot( alpha.T, numpy.dot( H_hat, alpha ) ) - numpy.dot( alpha.T, one_vect )
        grad_L_d = numpy.dot( H_hat, alpha ) - one_vect

        v = numpy.array((L_d, grad_L_d), dtype=object)

        return v
    return L_dual

def radialSVM(DTR, LTR, DTE, params):
    K, C, gamma = params[0], params[1], params[2]
    
    x0 = numpy.zeros((DTR.shape[1]), dtype='int32')
    H_hat = ml.compute_H_hat2(DTR, LTR, K**2, None, None, gamma)

    boundaries = []
    for i in range(DTR.shape[1]):
        boundaries.append((0, C))

    alpha, f, dictionary = fmin_l_bfgs_b(L_dual_wrapper(H_hat), x0, bounds=boundaries, factr=1.0)

    S = ml.compute_score(alpha, DTR, LTR, DTE, K**2, None, None, gamma)

    return list(S)

In [10]:
def kfold(D, L, fold, params, app):
    error = 0
    N = int(D.shape[1]/fold) #numero di elementi per ogni fold
    numpy.random.seed(0) #imposto il seed del generatore di numeri casuali -> in tal modo genererò la stessa sequenza di numeri casuali aventi seed uguale
    indexes = numpy.random.permutation(D.shape[1]) #genero una sequenza di numeri casuali che vanno da 0 al num_di_campioni
    
    LTE_final = []
    llr_final = []
    for j in range(fold):
        test_indexes = indexes[(j*N):((j+1)*N)] #selezioni gli indici che identificano i campioni (casuali) del test set
        if(j > 0): #se il test set non è preso dalla prima fold (--> il test set è una fold intermedia o l'ultima fold)
            left_indexes = indexes[0:(j*N)] #allora prendo tutti gli indici che stanno a sinistra di tale fold
        else: #se il test set è preso dalla prima fold
            right_indexes = indexes[((j+1)*N):] #prendo tutti gli indici a destra della prima fold

        if(j == 0): #se il test set è preso dalla prima fold
            train_indexes = right_indexes #assegno agli indici di training quelli che stanno a destra della prima fold
        elif(j == fold-1): #se il test set è preso dall'ultima fold
            train_indexes = left_indexes #assegno agli indici di training quelli che stanno a sinistra dell'ultima fold
        else: #in questo caso il test set è preso da una fold intermedia
            train_indexes = numpy.hstack((left_indexes, right_indexes)) #pertanto assegno agli indici di training quelli appartenenti alle fold di sinistra e di destra

        DTR = D[:, train_indexes]  #definisco insieme di training e di testing
        LTR = L[train_indexes]
        DTE = D[:, test_indexes]
        LTE = L[test_indexes]
        LTE_final.extend(LTE)
        llr_final.extend(radialSVM(DTR, LTR, DTE, params))
        
    CM = ml.compute_optimal_B_decision(app, llr_final, LTE_final)
    sensitivity = (1-ml.compute_FNR(CM)) # TPR
    specificity = (1-ml.compute_FPR(CM)) # TNR
        
    app_bayes_risk=ml.compute_Bayes_risk(CM, app)
    DCF = ml.compute_norm_Bayes(app_bayes_risk, app)

    minDCF= ml.compute_min_DCF(llr_final, app, LTE_final)
    error = 1-(CM[0, 0]+CM[1,1])/(len(LTE_final))

    print("\-/ \-/ \-/ \-/ \-/ ")
    print("Radial Basis SVM error:", error)
    print(app,"DCF:", DCF, "minDCF:", minDCF)
    print('Sensitivity (TPR):', sensitivity, ' Specificity (TNR): ', specificity)
    print('CM\n', CM)
    print("/-\ /-\ /-\ /-\ /-\ ")

In [11]:
kfold(Data, labels, 5, [0, 1, 1], [0.5, 1, 1])
kfold(Data, labels, 5, [0, 1, 10], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Radial Basis SVM error: 0.015350140056022421
[0.5, 1, 1] DCF: 0.10009718362866174 minDCF: 0.10009718362866174
Sensitivity (TPR): 0.9074299634591961  Specificity (TNR):  0.9924728529121422
CM
 [[8043.   76.]
 [  61.  745.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Radial Basis SVM error: 0.022184873949579798
[0.5, 1, 1] DCF: 0.18534312764752495 minDCF: 0.1051764034662662
Sensitivity (TPR): 0.8209500609013398  Specificity (TNR):  0.9937068114511353
CM
 [[8053.  147.]
 [  51.  674.]]
/-\ /-\ /-\ /-\ /-\ 


In [12]:
kfold(Data, labels, 5, [1, 1, 1], [0.5, 1, 1])
kfold(Data, labels, 5, [1, 1, 10], [0.5, 1, 1])

\-/ \-/ \-/ \-/ \-/ 
Radial Basis SVM error: 0.076750700280112
[0.5, 1, 1] DCF: 0.8343483556638246 minDCF: 0.8003455083909181
Sensitivity (TPR): 0.1656516443361754  Specificity (TNR):  1.0
CM
 [[8104.  685.]
 [   0.  136.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Radial Basis SVM error: 0.039887955182072776
[0.5, 1, 1] DCF: 0.4336175395858709 minDCF: 0.37937852377076087
Sensitivity (TPR): 0.5663824604141291  Specificity (TNR):  1.0
CM
 [[8104.  356.]
 [   0.  465.]]
/-\ /-\ /-\ /-\ /-\ 


In [13]:
data_g = ml.gaussianize(Data)
data_z = ml.z_normalization(Data)

In [14]:
kfold(data_g, labels, 5, [0, 1, 1], [0.5, 1, 1])
kfold(data_g, labels, 5, [1, 1, 10], [0.5, 1, 1])
kfold(data_g, labels, 5, [1, 1, 1], [0.5, 1, 1])
kfold(data_g, labels, 5, [1, 1, 10], [0.5, 1, 1])

KeyboardInterrupt: 

In [ ]:
kfold(data_z, labels, 5, [0, 1, 1], [0.5, 1, 1])
kfold(data_z, labels, 5, [1, 1, 10], [0.5, 1, 1])
kfold(data_z, labels, 5, [1, 1, 1], [0.5, 1, 1])
kfold(data_z, labels, 5, [1, 1, 10], [0.5, 1, 1])